# Assignment 07

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/NLP/Assignment-07

/content/drive/MyDrive/NLP/Assignment-07


In [4]:
df = pd.read_csv('bbc-text.csv')

In [5]:
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


Checking for null values

In [6]:
df.isnull().any()

,0
category,False
text,False


Check if the data is balanced

In [7]:
df['category'].value_counts()

,count
category,
sport,511
business,510
politics,417
tech,401
entertainment,386


### Convert categories to intergers

In [8]:
df['category'] = df['category'].map({'sport': 0, 'business': 1, 'politics': 2, 'tech': 3, 'entertainment': 4})

### splitting data

In [9]:
features = df['text']
labels = df['category']

In [10]:
train_data, test_data, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state = 42)

## Import and Init vectorizers

In [13]:
from Word2VecVectorizer import Word2VecVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [12]:
vectorizer = Word2VecVectorizer() # Create the instance of Word2Vec vectorizer

Loading in word vectors...
Finished loading in word vectors


In [14]:
# Initialize vectorizers
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()

In [15]:
# Applying Word2Vecvectorizer
word_2_vec_X_train = vectorizer.fit_transform(train_data)
word_2_vec_X_test = vectorizer.transform(test_data)

# Applying CountVectorizer
count_X_train = count_vectorizer.fit_transform(train_data).toarray()
count_X_test = count_vectorizer.transform(test_data).toarray()

# Applying TfidfVectorizer
tfidf_X_train = tfidf_vectorizer.fit_transform(train_data).toarray()
tfidf_X_test = tfidf_vectorizer.transform(test_data).toarray()

Numer of samples with no words found: 0 / 1780
Numer of samples with no words found: 0 / 445


In [18]:
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(input_dim,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(np.unique(train_labels)), activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

## Trainning

In [19]:
# Train with Word2VecVectorizer
print("Training with Word2VecVectorizer...")
model_word2vec = create_model(word_2_vec_X_train.shape[1])
history_word2vec = model_word2vec.fit(word_2_vec_X_train, train_labels, epochs=10, batch_size=32, validation_data=(word_2_vec_X_test, test_labels))

Training with Word2VecVectorizer...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 94ms/step - accuracy: 0.3109 - loss: 1.5549 - val_accuracy: 0.5708 - val_loss: 1.2446
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5908 - loss: 1.1428 - val_accuracy: 0.8315 - val_loss: 0.6982
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8006 - loss: 0.6924 - val_accuracy: 0.9191 - val_loss: 0.3874
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8785 - loss: 0.4552 - val_accuracy: 0.9371 - val_loss: 0.2600
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9011 - loss: 0.3628 - val_accuracy: 0.9416 - val_loss: 0.2013
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9024 - loss: 0.3060 - val_accuracy: 0.9506 - val_loss: 0.1730
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9311 - loss: 0.2421 - val_accuracy: 0.9416 - val_loss: 0.1610
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9325 - loss: 0.2299 - val_accuracy: 0.9551 - val_loss

In [20]:
# Train with CountVectorizer
print("Training with CountVectorizer...")
model_count = create_model(count_X_train.shape[1])
history_count = model_count.fit(count_X_train, train_labels, epochs=10, batch_size=32, validation_data=(count_X_test, test_labels))

Training with CountVectorizer...
Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 112ms/step - accuracy: 0.5072 - loss: 1.2495 - val_accuracy: 0.9506 - val_loss: 0.2528
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9287 - loss: 0.2707 - val_accuracy: 0.9663 - val_loss: 0.1099
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9647 - loss: 0.1348 - val_accuracy: 0.9640 - val_loss: 0.0975
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9761 - loss: 0.0672 - val_accuracy: 0.9730 - val_loss: 0.0987
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9881 - loss: 0.0541 - val_accuracy: 0.9753 - val_loss: 0.0997
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9810 - loss: 0.0590 - val_accuracy: 0.9663 - val_loss: 0.1428
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9876 - loss: 0.0662 - val_accuracy: 0.9685 - val_loss: 0.1064
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9967 - loss: 0.

In [21]:
# Train with TfidfVectorizer
print("\nTraining with TfidfVectorizer...")
model_tfidf = create_model(tfidf_X_train.shape[1])
history_tfidf = model_tfidf.fit(tfidf_X_train, train_labels, epochs=10, batch_size=32, validation_data=(tfidf_X_test, test_labels))


Training with TfidfVectorizer...
Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 59ms/step - accuracy: 0.4231 - loss: 1.5404 - val_accuracy: 0.8989 - val_loss: 0.9626
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8815 - loss: 0.7415 - val_accuracy: 0.9685 - val_loss: 0.2123
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9792 - loss: 0.1725 - val_accuracy: 0.9640 - val_loss: 0.1080
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9944 - loss: 0.0640 - val_accuracy: 0.9730 - val_loss: 0.0882
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9978 - loss: 0.0397 - val_accuracy: 0.9730 - val_loss: 0.0801
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9971 - loss: 0.0239 - val_accuracy: 0.9753 - val_loss: 0.0711
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9987 - loss: 0.0161 - val_accuracy: 0.9775 - val_loss: 0.0669
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9993 - loss: 0.0119

## Evaluation

In [22]:
print("\nEvaluation with Word2VecVectorizer:")
model_word2vec.evaluate(word_2_vec_X_test, test_labels)

print("\nEvaluation with CountVectorizer:")
model_count.evaluate(count_X_test, test_labels)

print("\nEvaluation with TfidfVectorizer:")
model_tfidf.evaluate(tfidf_X_test, test_labels)


Evaluation with Word2VecVectorizer:
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9405 - loss: 0.1507

Evaluation with CountVectorizer:
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9624 - loss: 0.1456

Evaluation with TfidfVectorizer:
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9681 - loss: 0.0782


[0.070262610912323, 0.9752808809280396]

# Conclusion

TfidfVectorizer is performing better for this dataset